In [11]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
OCI_ACCESS_KEY_ID = "f12a940090e79f11b9c8686f50d1f9abc357307f"
OCI_SECRET_ACCESS_KEY = "UOs9CI56ZvGIETioUV9ijqxKgtKU1a/fT7jg4Qvkq8c="
OCI_REGION = 'uk-london-1'
OCI_NAMESPACE = 'lrqgbz9z6zlj'
BUCKET_NAME = 'london-property-sales-price'

In [19]:
db_url = "jdbc:postgresql://pgdatabase:5432/prova"
db_properties = {
    "user": "root",
    "password": "root",
    "driver": "org.postgresql.Driver"
}

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('PySparkOCIConnection') \
    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.506') \
    .config('spark.hadoop.fs.s3a.endpoint', f'https://{OCI_NAMESPACE}.compat.objectstorage.{OCI_REGION}.oraclecloud.com') \
    .config('spark.hadoop.fs.s3a.access.key', OCI_ACCESS_KEY_ID) \
    .config('spark.hadoop.fs.s3a.secret.key', OCI_SECRET_ACCESS_KEY) \
    .config('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .config('spark.hadoop.fs.s3a.path.style.access', 'true') \
    .config('spark.hadoop.fs.s3a.connection.ssl.enabled', 'true') \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.7.4.jar") \
    .getOrCreate()

In [5]:
file_path = f's3a://{BUCKET_NAME}/chunks/*.parquet'

df = spark.read.parquet(file_path, header=True, inferSchema=True)

In [17]:
grouped = df.groupby(F.col('DISTRICT')).agg(F.sum('PRICE').alias('somma'))

In [20]:
grouped.write.jdbc(url=db_url, table="prova1", mode="overwrite", properties=db_properties)


In [7]:
spark.stop()